In [1]:
import json
import numpy as np

def pre_process(lst,length):
    to_lst = np.zeros(length)
    for ele in lst:
        to_lst[ele] = 1
    return to_lst

x = []
y = []
data = json.load(open("train.json"))
t = 0
for _ , ele in data.items():
    to_lst = []
    to_lst2 = []
    labels = []
    for k, v in ele.items():
        if k == 'venue':
            if v != '':
                to_lst.append(v)
            else:
                to_lst.append(0)
        elif k == 'keywords':
            temp = pre_process(v,500)
            to_lst += temp.tolist()
        elif k == 'year':
            to_lst.append(v)
        elif k == 'author':
            t += len(v)
            if len(v) == 1:
                to_lst2.append(np.zeros(2302).tolist())
                labels.append(v[0])
            else:
                for i in range(len(v)):
                    co = v[:i] + v[i+1:]
                    au = v[i]
                    temp = pre_process(co,2302)
                    to_lst2.append(temp.tolist())
                    labels.append(au)
    for i in range(len(to_lst2)):
        x.append(to_lst+to_lst2[i])
        y.append(labels[i])
                    
            
    
print("Number of training data =", len(x))
print("Number of labels =", len(y))



Number of training data = 48000
Number of labels = 48000


In [2]:
data_t = json.load(open("test.json"))
x_t = []
y_t = []
for _ , ele in data_t.items():
    to_lst = []
    labels = []
    for k, v in ele.items():
        if k == 'venue':
            if v != '':
                to_lst.append(v)
            else:
                to_lst.append(0)
        elif k == 'keywords':
            temp = pre_process(v,500)
            to_lst += temp.tolist()
        elif k == 'year':
            to_lst.append(v)
        elif k == 'coauthor':
            temp = pre_process(v,2302)
            to_lst += temp.tolist()
        elif k == 'target':
            y_t.append(v)
    x_t.append(to_lst)

print("Number of testing data =", len(x_t))
print("Number of target labels =", len(y_t))

Number of testing data = 2000
Number of target labels = 2000


In [3]:
from sklearn.model_selection import train_test_split
x_train, x_dev, y_train, y_dev = train_test_split(x, y, test_size=0.1, random_state = 0,stratify=y)


In [4]:
from collections import Counter
c1 = Counter(y_train)
c2 = Counter(y_train)
print(len(c1))
print(len(c2))

2302
2302


In [4]:
from sklearn.decomposition import PCA
from sklearn import preprocessing
normalizer = preprocessing.MinMaxScaler().fit(x)
norm_x = normalizer.transform(x)
norm_x_dev = normalizer.transform(x_dev)
#normalizer = preprocessing.StandardScaler().fit(x_train)
#norm_x = normalizer.transform(x_train)
#norm_x_dev = normalizer.transform(x_dev)
pca = PCA(n_components=0.99)
pca.fit(norm_x)
print(pca.n_components_)

1637


In [5]:
pca_x = pca.transform(norm_x).tolist()
print(pca_x[0])
pca_x_dev = pca.transform(norm_x_dev).tolist()

[-0.08045089572087126, 0.3014299191993138, 0.539354273998595, -0.8782407808670396, -0.5669220994693382, -0.06169900499887903, 0.6456626046083188, -0.6435304374141676, 0.11891081900826556, -0.13101992395770304, -0.08689285763859983, 1.0585745926473353, -0.18513644811859975, 0.7824641693457958, 0.2806178070051963, 0.4522451036806366, -0.09805537494548879, -0.35595495688870366, 1.0900122355779152, 0.40891626119557334, -0.5871153187249859, -0.8017770439922638, -0.3187568787988082, -0.3796963782914236, -0.005686175364555111, 0.4725356692386169, -0.4111396402584474, 0.07497287095212793, -0.40433569455494456, -0.12450293800204336, -0.11045329480503441, -0.24840456646889508, 0.10955374517449067, -0.3573381564163861, 0.8040107530544435, 0.3742531508082637, 0.2291005281957006, -0.2531653854345808, -0.05033004050126785, 0.071945405453334, 0.31443091788622135, -0.2701011976526935, -0.016977851019366948, 0.06291412597623644, -0.16040705275584669, 0.8795305836011695, 0.00112744390006158, -0.07883266

In [10]:
from keras.models import Sequential
from keras.layers import Dense
import keras
import tensorflow as tf
model2 = Sequential()
model2.add(Dense(500, input_dim=pca.n_components_, activation='relu'))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(2302, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=)
#t = tf.keras.metrics.AUC(multi_label = True)
model2.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 500)               819000    
_________________________________________________________________
dense_7 (Dense)              (None, 100)               50100     
_________________________________________________________________
dense_8 (Dense)              (None, 2302)              232502    
Total params: 1,101,602
Trainable params: 1,101,602
Non-trainable params: 0
_________________________________________________________________


In [11]:

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)
model2.fit(pca_x, y_train, epochs=20, verbose=True, validation_data=(pca_x_dev, y_dev), batch_size=100,callbacks = [callback])
#model2.fit(norm_pca_x, y_train, epochs=10, verbose=True, validation_data=(norm_pca_x_dev, y_dev), batch_size=100,callbacks = [callback])
#loss, accuracy = model2.evaluate(pca_x_dev, y_dev, verbose=False)
#print("\nTesting Accuracy:  {:.4f}".format(accuracy))

Epoch 1/20
432/432 [==============================] - 16s 37ms/step - loss: 7.2997 - accuracy: 0.0143 - val_loss: 6.6729 - val_accuracy: 0.0327
Epoch 2/20
432/432 [==============================] - 16s 38ms/step - loss: 5.6617 - accuracy: 0.0970 - val_loss: 5.1928 - val_accuracy: 0.1319
Epoch 3/20
432/432 [==============================] - 16s 36ms/step - loss: 3.9925 - accuracy: 0.2455 - val_loss: 4.5951 - val_accuracy: 0.1948
Epoch 4/20
432/432 [==============================] - 16s 38ms/step - loss: 3.0060 - accuracy: 0.3723 - val_loss: 4.4470 - val_accuracy: 0.2185
Epoch 5/20
432/432 [==============================] - 17s 38ms/step - loss: 2.3592 - accuracy: 0.4743 - val_loss: 4.4643 - val_accuracy: 0.2300
Epoch 6/20
432/432 [==============================] - 16s 36ms/step - loss: 1.8668 - accuracy: 0.5661 - val_loss: 4.5893 - val_accuracy: 0.2373
Epoch 7/20
432/432 [==============================] - 16s 37ms/step - loss: 1.4768 - accuracy: 0.6465 - val_loss: 4.7247 - val_accuracy:

In [36]:
norm_x_t = normalizer.transform(x_t)
pca_x_t = pca.transform(norm_x_t).tolist()
pre_y_test = model2.predict(pca_x_t)

In [37]:
pre_lst = []
for i in range(len(y_t)):
    pre_lst.append(pre_y_test[i][y_t[i]])
print(len(pre_lst))
print(pre_lst[0])

2000
2.2161212e-05


In [38]:
import pandas as pd
index = [i for i in range(2000)]
dataframe = pd.DataFrame({'Id':index,'Predicted':pre_lst})
#这个地址你们改成自己电脑的
dataframe.to_csv("C:/Users/ZUQING/Desktop/PREDICT3.csv",index=0)